**CIC-Collection Cleaning: CIC-IDS2018**

This notebook presents the preprocessing done on CIC-IDS2018:
*   read and store path of all file
*   read all CSV file
*   drop column
*   replace inf value
*   drop nan rows
*   drop duplicate rows
*   concatenate all cleaning file
*   normalize
* Merge similar attcks and remove zero and non relvant coloumns

In [ ]:
!pip install pandas==1.5.3
!pip install dask-ml

In [ ]:
#import statement
import numpy as np
import os
import gc
import pandas as pd
from datetime import datetime
#from sklearn.preprocessing import MinMaxScaler
from dask_ml.preprocessing import MinMaxScaler

#pd.show_versions()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#def to select BENIGN, Dos and DDOs labeled row

def label_info(df):
  #print (df.dtypes)
  labels_name = pd.unique(df["Label"])
  labels = df.groupby(['Label'])['Label']
  labels_count = labels.count()
  print('The number of output labels is {}:', len(labels_count))
  print('='*50)
  print('Label distribution:', labels_count)
  return labels_name

In [ ]:
path = path= '/content/drive/MyDrive/dataset_zipped-20230608T100425Z-005/dataset_zipped/CSE-CIC-2018/cleaning_file/CICIDS2018_preprocessing_binary.parquet'
df = pd.read_parquet(path)
print (df.shape)

(7384563, 67)


In [ ]:
#def to remove columns

#def drop_columns(df, column_to_drop):
df = df.drop(columns=column_to_drop, axis=1)
print("REMOVE {} COLUMNS from dataframe, NEW COLUMN number: {}".format(len(column_to_drop), len(df.columns)))
print ("Shape after preprocessing --> ", df.shape)
  #return df

NameError: ignored

In [ ]:
#def to drop duplicate rows

def drop_dup(df):
  print("SEARCHING Duplicate Rows on Dataframe")
  initial_row = len(df['Label'])
  df = df.drop_duplicates()
  final_row = len(df['Label'])
  print("DROPPED {} Duplicated rows, new shape is {}".format(initial_row-final_row, df.shape))
  return df

In [ ]:
#def to replace inf value

def replace_inf(df):
  print("SEARCHING Inf value on Dataframe") #with shape: ", df.compute().shape)
  count_inf = np.isinf(df.select_dtypes(include=['float64','int64'])).values.sum()
  print("--> REACHED {} number of Inf value".format(count_inf))
  df = df.replace([np.inf, -np.inf], -1)
  print("REPLACED Inf values by -1")
  return df

In [ ]:
#drop nan - when apply normally dropna dask built-in function not properly work

def drop_na(df):
  print("SEARCHING Nan Rows on Dataframe") #with shape: ", df.compute().shape)
  count_na = df.isna().sum().sum()
  print("--> REACHED {} of NaN Rows".format(count_na))
  df = df.dropna()
  print("DROPPED NaN rows on Dataframes") #with actual shape: ", df.compute().shape)
  return df


In [ ]:
#normalize with min_max in range 0-1

def min_max_norm(df):
  print ("START normalization...")

  scaler = MinMaxScaler()
  df_norm = scaler.fit_transform(df.select_dtypes(include=['int64', 'float64']))

  for k in df_norm.columns:
    df[k] = df_norm[k]

  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#read and store path of all file
#path = '/content/drive/MyDrive/dataset_zip/CSE-CIC-2018/cleaning_file'
#file_to_read = [0,1,2,3]
pd.set_option("display.max_row", None, "display.max_columns", 200, 'display.width', 2000)

path = '/content/drive/MyDrive/dataset_zip/CSE-CIC-2018'
file_to_read = [1,2,3,4]
dspaths = []
labels = []
df_total = pd.DataFrame()
column_to_drop = ["Dst Port", "Protocol", "Timestamp"]
column_to_drop_extended= ["Flow ID", "Src IP", "Src Port", "Dst IP", "Dst Port", "Protocol", "Timestamp"]
column_to_drop_0_mean = ['Bwd PSH Flags','Fwd URG Flags','Bwd URG Flags','CWE Flag Count','Fwd Byts/b Avg','Fwd Pkts/b Avg','Fwd Blk Rate Avg','Bwd Byts/b Avg','Bwd Pkts/b Avg','Bwd Blk Rate Avg']

#verify if work is yet done

print ("List of files in {}: \n".format(path))
for dirname, _, filenames in os.walk(path):
  for filename in filenames:
    if filename.endswith(".csv"):
      abs_path = os.path.join(dirname, filename)
      dspaths.append(abs_path)
      print ("{} size = {} MB".format(abs_path, f'{os.path.getsize(abs_path)/(1024*1024):.2f}'))
print ("Total File REACHED --> ", len(dspaths))

if (len(dspaths)!=0):
  for dsp in dspaths:
    if dspaths.index(dsp) in file_to_read:
      print ("\nSTART READ dataframe {} --> {}".format(dspaths.index(dsp), dsp))
      df = pd.read_csv(dsp, engine='pyarrow')
      #df.info()
      labels.append(label_info(df))

      if path == '/content/drive/MyDrive/dataset_zip/CSE-CIC-2018':
        print ("Shape before preprocessing --> ", df.shape)
        if len(df.columns) != 80:
          df = drop_columns(df, column_to_drop_extended)
        else:
          df = drop_columns(df, column_to_drop)
        df = drop_dup(df)
        df = replace_inf(df)
        df = drop_na(df)
        print ("Shape after preprocessing --> ", df.shape)
        path_file = os.path.join('/content/drive/MyDrive/dataset_zip/CSE-CIC-2018/cleaning_file/', str(dspaths.index(dsp)))
        print ("Write on --> ", path_file)
        df.to_csv(path_file)

      else:
        print ("Dataframe is yet preprocessed --> ", df.shape)

      df = drop_columns(df, column_to_drop_0_mean)
      #print (df.describe())
      print ("CONCAT FILE on df_Total")
      df_total = pd.concat([df_total, df])

else:
    exit()

#print (df_total.head(10))
gc.collect()
#del df

List of files in /content/drive/MyDrive/dataset_zip/CSE-CIC-2018: 

Total File REACHED -->  0


101

In [ ]:
#print (df_total.dtypes)
#print (df_total.describe())
print ("df_total shape--> {}".format(df_total.shape))

#print ("Memory usage ", df_total.memory_usage(deep=True))

NameError: ignored

In [ ]:
#normalization
print ("Before scaling : \n", df_total.describe())
#print (df_total.dtypes)
df_total = min_max_norm(df_total.drop("",axis=1))
print ("_"*50)
print ("After scaling : \n", df_total.describe())

NameError: ignored

In [ ]:
#divide label from other data
Ddos_attack = ['DoS attacks-GoldenEye', 'DoS attacks-Slowloris', 'DDoS attacks-LOIC-HTTP', 'DDOS attack-HOIC', 'DDOS attack-LOIC-UDP', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk']
df_labels_binary = pd.DataFrame()
df_labels_binary["Label"] = df_total["Label"]
df_labels_binary = df_labels_binary.replace(Ddos_attack, 1)
df_labels_binary = df_labels_binary.replace("Benign", 0)
print('='*50)

print('The number of output labels is {}:', df_labels_binary.groupby(['Label'])['Label'].count())

NameError: ignored

In [ ]:
#export to parquet binary
print ("START EXPORT TO PARQUET FILE...")
name = 'CICIDS2018_preprocessing_binary.parquet'
pd.concat([df_total.drop(["Label"], axis=1), df_labels_binary], axis=1).to_parquet(path + "/" + name)
print("\nFile {} CREATED in {}".format(name, path))


START EXPORT TO PARQUET FILE...

File CICIDS2018_preprocessing_binary.parquet CREATED in /content/drive/MyDrive/dataset_zip/CSE-CIC-2018/cleaning_file


In [ ]:
print ("Start READ parquet file")
df = pd.read_parquet(path + "/" + name)
print (df.head(10))

Start READ parquet file
   Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  Fwd Pkt Len Std  Bwd Pkt Len Max  Bwd Pkt Len Min  Bwd Pkt Len Mean  Bwd Pkt Len Std   Flow Byts/s   Flow Pkts/s  Flow IAT Mean  Flow IAT Std  Flow IAT Max  Flow IAT Min   Fwd IAT Tot  Fwd IAT Mean   Fwd IAT Std   Fwd IAT Max   Fwd IAT Min  Bwd IAT Tot  Bwd IAT Mean  Bwd IAT Std  Bwd IAT Max   Bwd IAT Min  Fwd PSH Flags  Fwd Header Len  Bwd Header Len    Fwd Pkts/s    Bwd Pkts/s  Pkt Len Min  Pkt Len Max  Pkt Len Mean  Pkt Len Std  Pkt Len Var  FIN Flag Cnt  SYN Flag Cnt  RST Flag Cnt  PSH Flag Cnt  ACK Flag Cnt  URG Flag Cnt  ECE Flag Cnt  Down/Up Ratio  Pkt Size Avg  Fwd Seg Size Avg  Bwd Seg Size Avg  Subflow Fwd Pkts  Subflow Fwd Byts  Subflow Bwd Pkts  Subflow Bwd Byts  Init Fwd Win Byts  Init Bwd Win Byts  Fwd Act Data Pkts  Fwd Seg Size Min  Active Mean  Active Std  Active Max  Active Min  Idle Mean      Idle Std  Idle Max  I

In [ ]:
#one_hot encoding
print ("START OHE...\n")
labels = df_total['Label'].unique()
print (df_total.groupby(['Label'])['Label'].count()) #count Label
#print (df_total['Label'].head(50))
df_enc = pd.get_dummies(df_total["Label"])
#print (df_enc.head(50))

START OHE...

Label
Benign                      6412040
DDOS attack-HOIC             198861
DDOS attack-LOIC-UDP           1730
DDoS attacks-LOIC-HTTP       575364
DoS attacks-GoldenEye         41406
DoS attacks-Hulk             145199
DoS attacks-SlowHTTPTest         55
DoS attacks-Slowloris          9908
Name: Label, dtype: int64
    Benign  DDOS attack-HOIC  DDOS attack-LOIC-UDP  DDoS attacks-LOIC-HTTP  DoS attacks-GoldenEye  DoS attacks-Hulk  DoS attacks-SlowHTTPTest  DoS attacks-Slowloris
0        1                 0                     0                       0                      0                 0                         0                      0
1        1                 0                     0                       0                      0                 0                         0                      0
2        1                 0                     0                       0                      0                 0                         0                      0
3     

In [ ]:
#export to parquet OHE
print ("START EXPORT TO PARQUET FILE...")
name = 'CICIDS2018_preprocessing_OHE.parquet'
pd.concat([df_total.drop(["Label"], axis=1), df_enc], axis=1).to_parquet(path + "/" + name)
print("\nFile {} CREATED in {}".format(name, path))


START EXPORT TO PARQUET FILE...

File CICIDS2018_preprocessing_OHE.parquet CREATED in /content/drive/MyDrive/dataset_zip/CSE-CIC-2018/cleaning_file


In [ ]:
print ("Start READ parquet file")
df = pd.read_parquet(path + "/" + name)
print (df.head(10))

Start READ parquet file
   Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  Fwd Pkt Len Std  Bwd Pkt Len Max  Bwd Pkt Len Min  Bwd Pkt Len Mean  Bwd Pkt Len Std   Flow Byts/s   Flow Pkts/s  Flow IAT Mean  Flow IAT Std  Flow IAT Max  Flow IAT Min   Fwd IAT Tot  Fwd IAT Mean   Fwd IAT Std   Fwd IAT Max   Fwd IAT Min  Bwd IAT Tot  Bwd IAT Mean  Bwd IAT Std  Bwd IAT Max   Bwd IAT Min  Fwd PSH Flags  Fwd Header Len  Bwd Header Len    Fwd Pkts/s    Bwd Pkts/s  Pkt Len Min  Pkt Len Max  Pkt Len Mean  Pkt Len Std  Pkt Len Var  FIN Flag Cnt  SYN Flag Cnt  RST Flag Cnt  PSH Flag Cnt  ACK Flag Cnt  URG Flag Cnt  ECE Flag Cnt  Down/Up Ratio  Pkt Size Avg  Fwd Seg Size Avg  Bwd Seg Size Avg  Subflow Fwd Pkts  Subflow Fwd Byts  Subflow Bwd Pkts  Subflow Bwd Byts  Init Fwd Win Byts  Init Bwd Win Byts  Fwd Act Data Pkts  Fwd Seg Size Min  Active Mean  Active Std  Active Max  Active Min  Idle Mean      Idle Std  Idle Max  I